In [15]:
import requests 
import copy


s = {
    "baseURL": 'http://192.168.1.140:3001/',
    'id': "5bc59e192817116e84bdd831",
    'currentPoint': False
}


def getFunc(nodes):
    for key, value in nodes.items():
        req = requests.get(s['baseURL']+'pfunction/'+value['id'])
        func = req.json()
        funcs[func['_id']] = func
        gr = func['pfunction']['graph']
        if (gr != {}):
            getFunc(gr['n'])

def enrich_graph(gr):
    for n in gr['n']:
        x = gr['n'][n]
        x['out'] = {}; 
        x['in'] = {};
        x['position'] = {'x': 0, 'y': 0};
    for e in gr['e']:
        if e[0] == 0:
            pass
        else:
            if e[1]  in gr['n'][e[0]]['out']:
                gr['n'][e[0]]['out'][e[1]].append([e[2], e[3]])
            else:
                gr['n'][e[0]]['out'][e[1]] = [[e[2], e[3]]]
        gr['n'][e[2]]['in'][e[3]] = [e[0], e[1]]
    add_nodesedges(funcs, gr)
    visitors = [enrich]
    positionNodes(gr, list(gr['n'].keys()), [0], visitors, 0)
    s['currentPoint'] = False
    return gr

    

def add_nodesedges(context, gr):
    added_nodes = {}
    added_edges = []
    step_out = 4000
    step_in = 3000
    for n in gr['n']:
        x = gr['n'][n]
        for idx1, value1 in enumerate(context[x['id']]['pfunction']['gapi']['outputs'], start=1):
            if (idx1 not in x['out'] and x['id'] != s['id']):
                no = { 
                    'i': step_out,  
                    'svg_id': "", 
                    'id': s['id'] + str(step_out), 
                    'in':{1: [x['i'], idx1]}, 
                    'out':{}, 
                    'position': {'x': 0, 'y':0} ,
                    'edges': []
                  }
                typing  = value1 #['type']
                funcs[s['id'] + str(step_out)]  = {"_id": s['id'] + str(step_out),
                   "pclassid":"5c95397d4212cc40afeec914",
                   "pfunction":{"signature":"io", "source": "f : f", 
                    "graph":{}, "gapi":{"constant": True,"inputs":[{"name":typing['name'],"type": typing['type']}],
                    "name":"io","outputs":[{"name":"o","type": typing['type']}],"payable": False,
                    "stateMutability":"view","type":"function"},"chainids":["3"]},"tags":["pipeline","ethpm"],
                    "timestamp":""}
                added_nodes[step_out]  = no
                added_edges.append([x['i'], idx1, step_out, 1])
                x['out'][idx1]= [[step_out, 1]]
                step_out= step_out+1
    for n in gr['n']:
        x = gr['n'][n]
        for idx1, value1 in enumerate(context[x['id']]['pfunction']['gapi']['inputs'], start=1):
            if (idx1 not in x['in'] and x['id'] != s['id']):
                no = { 
                    'i': step_out,  
                    'svg_id': "", 
                    'id': s['id'] + str(step_in), 
                    'in': {}, 
                    'out': {1:[[x['i'], idx1]]},
                    'position': {'x': 0, 'y':0} ,
                    'edges': []
                  }
                typing  = value1 #['type']
                funcs[s['id'] + str(step_in)]  = {"_id": s['id'] + str(step_in),
                   "pclassid":"5c95397d4212cc40afeec914",
                   "pfunction":{"signature":"io", "source": "f : f", 
                    "graph":{}, "gapi":{"constant": True,"inputs":[{"name":typing['name'],"type": typing['type']}],
                    "name":"io","outputs":[{"name":"i","type": typing['type']}],"payable": False,
                    "stateMutability":"view","type":"function"},"chainids":["3"]},"tags":["pipeline","ethpm"],
                    "timestamp":""}
                added_nodes[step_in]  = no
                added_edges.append([step_in, 1, x['i'], idx1])
                x['in'][idx1]= [step_in, 1]
                step_in= step_in+1
    gr['n'] = {**gr['n'], **added_nodes }
    gr['e'] = gr['e'] + added_edges


def difference(first, second):
    second = set(second)
    return [item for item in first if item not in second]

def inputNodesAreVisited(node, visited):
    for key in node['in']:
        if not (node['in'][key][0] in visited): 
            return False
    return True
    
def inputNodesAreDefined(node, defined, level):
    #print(defined, node['in'])
    if (level == 0):
        return False
    for key in node['in']:
        if node['in'][key][0] in defined:
            return True
        """if not u:
            return False;
        if not u[node['in'][key][0]]:
            return False; """
    return False

def enrich(na):
    if not s['currentPoint']: 
        s['currentPoint'] = {'x': 0, 'y': 1}
    s['currentPoint']['y'] = na['level']
    if (na['row'] == 0):
        s['currentPoint']['x'] = 0
    na['node']['position'] = copy.copy(s['currentPoint'])
    max1  = max( len(na['context'][na['node']['id']]['pfunction']['gapi']['inputs']), 
                  len(na['context'][na['node']['id']]['pfunction']['gapi']['outputs']))
    s['currentPoint']['x'] = s['currentPoint']['x']  + max1
    
def positionNodes(gr, unvisited, visited, visitors, level):
    if len(unvisited) == 0:
        return;
    row = 0
    visitedNow = []
    only_outputs = True
    for unvis in unvisited:
        if unvis < 4000:
            only_outputs = False
    for n in unvisited:
        if (inputNodesAreVisited(gr['n'][n], visited) or inputNodesAreDefined(gr['n'][n], gr['r'], level)) or (level >0 or n > 2000):
            for visitor in visitors:
                visitor({"node": gr['n'][n],  "level": level, "row": row, "context": funcs})
            row = row + 1
            if only_outputs or n < 4000:
                visitedNow.append(n)
    unvisited = difference(unvisited, visitedNow)
    visited = visited + visitedNow
    return positionNodes(gr, unvisited, visited, visitors, level+1)

def make_runtime(context, rich_graph):
    rows = {}
    rich_graph2 = copy.deepcopy(rich_graph)
    for x in rich_graph2['n']:
        if not rich_graph2['n'][x]['position']['y'] in rows:
            rows[rich_graph2['n'][x]['position']['y']] = []
        rows[rich_graph2['n'][x]['position']['y']].append(x)
    return {
    "rich_graph": rich_graph2,
    "runnable_graph": [value for (key, value) in sorted(rows.items())],
    "context": context,
    "runtime": { 0: rich_graph2['r']}
  }

def run_graph( runtime_graph, ins):
    outs = []
    ndx = 0
    arr_ins = list(ins.values())
    runtime = runtime_graph['runtime']
    runnable = runtime_graph['runnable_graph']
    rich = runtime_graph['rich_graph']
    
    for key in runnable[0]:
        runtime[key] = [arr_ins[ndx]]
        ndx= ndx+1
    ndx = 0
    for x in runnable:
        for y in runnable[ndx]:
            node  = rich['n'][y]
            contxt = runtime_graph['context'][node['id']]
            source = contxt['pfunction']['source']
            args=[]
            if  (1 in node['in']):
                port  =  0
                for key1 in node['in']:
                    x1 = node['in'][key1]
                    #print(key1, x1)
                    argt = runtime[x1[0]][x1[1]-1]
                    type  = contxt['pfunction']['gapi']['inputs'][port]['type']
                    if (type == "function"):
                        argt  = eval("lambda " + argt) # "lambda a, b : a + b" +  argt "a, b : a + b"
                    port  = port +1
                    args.append(argt)
            else:
                args  = runtime[y]
            if (contxt['pfunction']['gapi']['type'] == "function"):
                if (len(contxt['pfunction']['graph']) > 0):
                    ans = run_graph (make_runtime (indexed_func) (enrich_graph (indexed_func) (contxt['pfunction']['graph']))) (args);
                    ans = ans.values()
                else:
                    func = eval("lambda " + source);
                    ans = func(*args);
            else:
                pass
            runtime[y] = [ans]
            if y >=  4000:
                outs.append(ans)
        ndx = ndx + 1
    return outs;

def fun_graph( context, graph, ins):
    return run_graph (make_runtime (context, enrich_graph (graph)),ins)
    
#enrich_graph(graph)
#print(funcs, graph)

#print(make_runtime (funcs, enrich_graph (graph)))



In [46]:
graph = {
  "n": {
    1: { "i": 1, "id": "5dc551ca33634f14a7cca672"},
      #2: { "i": 2, "id": "5dc551ca33634f14a7cca672"},
  },
  "e": [
          [0,1,1,1]
       ],
  "r": ["h : h*11"]
}

funcs = {}
getFunc(graph['n'])

In [17]:

print(fun_graph(funcs, graph, {"d": [89, 8, 89]}) ) # "a": "a, b : a + b", 

[[979, 88, 979]]


In [47]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import json

#egraph = make_runtime (funcs, enrich_graph (graph))
#print(egraph)


    
def operate(gr):
    inputs = []
    outputs = []
    consts = []
    def execute_gr(v):
        ins = {}
        ind = 0
        for item in inputs:
            #print(item.value)
            try:
                i = json.loads("["+item.value+"]")[0]
            except:
                i = json.loads("[\""+item.value+"\"]")[0]
            ins[ind] = i
            ind = ind +1
        #print(ins)
        #outs = fun_graph(funcs, gr, ins)
        outs = run_graph(gr, ins)
        ind  = 0
        for item in outputs:
            #print(item.value)
            #ins[ind] = json.loads(item.value)
            item.value = str(outs[ind])
            ind = ind +1
    for key in gr['runnable_graph']:
        for key2 in key:
            #print(key2)
            port = funcs[gr['rich_graph']['n'][key2]['id']]['pfunction']['gapi']['outputs'][0]
            if key2 > 2999 and key2 <4000:
                #node
                #print(key2)
                inputs.append(widgets.Text(
                    value='',
                    placeholder='Input something',
                    description= port['name']+': '+port['type'],
                    disabled=False
                ))
            if key2 > 3999:
                outputs.append(widgets.Text(
                    value='',
                    placeholder='Output',
                    description= port['name']+': '+port['type'],
                    disabled=True
                ))
    for key in gr['runtime']:
        #print(gr['runtime'][key])
        consts.append(widgets.Text(
                    value = str(gr['runtime'][key]),
                    description='Constant:',
                    disabled=True
                )
        )
    exe = widgets.Button(
        description='Execute',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Execute now',
        icon='check'
    )
    exe.on_click(execute_gr)
    display(widgets.HBox([widgets.VBox(inputs + consts), widgets.VBox(outputs)]),exe)
    
operate(egraph)

Button(description='Execute', icon='check', style=ButtonStyle(), tooltip='Execute now')

In [32]:
print(egraph)


{'rich_graph': {'n': {1: {'i': 1, 'id': '5dc551ca33634f14a7cca672', 'out': {1: [[4000, 1]]}, 'in': {1: [0, 1], 2: [3000, 1]}, 'position': {'x': 0, 'y': 1}}, 2: {'i': 2, 'id': '5dc551ca33634f14a7cca672', 'out': {1: [[4001, 1]]}, 'in': {1: [3001, 1], 2: [3002, 1]}, 'position': {'x': 2, 'y': 1}}, 4000: {'i': 4000, 'svg_id': '', 'id': '5bc59e192817116e84bdd8314000', 'in': {1: [1, 1]}, 'out': {}, 'position': {'x': 0, 'y': 2}, 'edges': []}, 4001: {'i': 4001, 'svg_id': '', 'id': '5bc59e192817116e84bdd8314001', 'in': {1: [2, 1]}, 'out': {}, 'position': {'x': 1, 'y': 2}, 'edges': []}, 3000: {'i': 4002, 'svg_id': '', 'id': '5bc59e192817116e84bdd8313000', 'in': {}, 'out': {1: [[1, 2]]}, 'position': {'x': 2, 'y': 0}, 'edges': []}, 3001: {'i': 4002, 'svg_id': '', 'id': '5bc59e192817116e84bdd8313001', 'in': {}, 'out': {1: [[2, 1]]}, 'position': {'x': 3, 'y': 0}, 'edges': []}, 3002: {'i': 4002, 'svg_id': '', 'id': '5bc59e192817116e84bdd8313002', 'in': {}, 'out': {1: [[2, 2]]}, 'position': {'x': 4, 'y

In [7]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return

slow = interactive(slow_function, {'manual': False}, i=widgets.FloatSlider(min=1e4, max=1e6, step=1e4))
display(slow)


interactive(children=(FloatSlider(value=10000.0, description='i', max=1000000.0, min=10000.0, step=10000.0), O…

In [ ]:
import types

dynf = types.FunctionType(compile('print("fgh")', 'dyn.py', 'exec'), {})
dynf()

In [ ]:
i = eval("lambda f : f")
i(9)